In [9]:
def process_prescales_data(input_file, output_file):
    with open(input_file, 'r') as file:
        lines = file.readlines()


    current_run_data = []
    consolidated_data = []


    for line in lines:
        if line.strip().startswith('+'):
            if current_run_data:
        
                consolidated_data.append(','.join(current_run_data))
                current_run_data = []  
            continue 

        parts = line.strip().strip('|').split('|')

        parts = [part.strip() for part in parts]

  
        if parts[0]:  # This means it's a new run line
            if current_run_data:
              
                consolidated_data.append(','.join(current_run_data))
                current_run_data = []
            
            current_run_data = parts[:6]  # Assuming the first 6 columns are fixed


        current_run_data.append(parts[-1])  # Use the last part which should be l1bit/prescval


    if current_run_data:
        consolidated_data.append(','.join(current_run_data))


    with open(output_file, 'w') as file:
        for line in consolidated_data:
            file.write(line + '\n')


input_file = 'prescales_2016.txt'
output_file = 'processed_prescales_2016.csv'
process_prescales_data(input_file, output_file)

In [1]:
import pickle

def extract_l1_configurations(input_file):
    l1_dict = {}
    with open(input_file, 'r') as file:
        for line in file:
            parts = line.strip().split(',')
            run_number = parts[0]
            l1_config = 'OR ' + line.split('OR ')[1].strip()

            if l1_config not in l1_dict:
                l1_dict[l1_config] = set()
            l1_dict[l1_config].add(run_number)

    # Convert sets to lists
    for key in l1_dict:
        l1_dict[key] = list(l1_dict[key])

    return l1_dict

# File paths
input_file = 'processed_prescales_2017.csv'
output_pickle = 'l1_configurations_2017.pickle'


l1_configurations = extract_l1_configurations(input_file)
with open(output_pickle, 'wb') as handle:
    pickle.dump(l1_configurations, handle, protocol=pickle.HIGHEST_PROTOCOL)

IndexError: list index out of range

In [16]:
import pickle


file_path = 'processed_prescales_2017.csv'

or_patterns = {}


with open(file_path, 'r') as file:
    for line in file:
        # Skip lines containing 'DST_HT410_PFScouting_v9/0'
        if 'DST_HT410_PFScouting_v9/0' in line:
            continue
        
        parts = line.strip().split(',')
        run_number = parts[0]
        
      
        or_index = line.find('OR')
        if or_index != -1:
            pattern = line[or_index + 3:].strip()  
            

            if pattern not in or_patterns:
                or_patterns[pattern] = []
            or_patterns[pattern].append(run_number)

output_pickle = 'l1_2017_patterns.pickle'
with open(output_pickle, 'wb') as handle:
    pickle.dump(or_patterns, handle, protocol=pickle.HIGHEST_PROTOCOL)

print(f"Dictionary saved to {output_pickle}")

Dictionary saved to l1_2017_patterns.pickle


In [15]:
import pickle

# Define the path to your CSV file
file_path = 'processed_prescales_2017_filtered.csv'

# Initialize a dictionary to hold all L1 configurations with a value of '1.00' and their corresponding run numbers
l1_config_runs = {}

# Open the file and process each line
with open(file_path, 'r') as file:
    for line in file:
        # Skip lines containing 'DST_HT410_PFScouting_v9/0'
        if 'DST_HT410_PFScouting_v9/0' in line:
            continue
        
        parts = line.strip().split(',')
        run_number = parts[0]
        
        # Process each part to find L1_* entries with '/1.00'
        for part in parts:
            if 'L1_' in part and '/1.00' in part:
                if part not in l1_config_runs:
                    l1_config_runs[part] = set()  # Use a set to ensure uniqueness
                l1_config_runs[part].add(run_number)

# Convert sets to lists for easier handling later
for key in l1_config_runs:
    l1_config_runs[key] = sorted(list(l1_config_runs[key]), key=int)

# Save the dictionary to a file using pickle
output_pickle = 'l1_configurations_2017.pickle'
with open(output_pickle, 'wb') as handle:
    pickle.dump(l1_config_runs, handle, protocol=pickle.HIGHEST_PROTOCOL)

print(f"Dictionary saved to {output_pickle}")

Dictionary saved to l1_configurations_2017.pickle
